## Importação das bibliotecas

In [ ]:
!python -m pip install xgboost

In [ ]:
import pandas as pd
from joblib import dump, load
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

## Puxando dataframe e dicionário com decodificação das variáveis categóricas

In [ ]:
%run Pre_Processamento.ipynb

In [ ]:
%store -r df_1

In [ ]:
df = df_1

In [ ]:
decoding_dict = load('decoding_dicts.joblib')

## Preparação dos dados

##### Agrupamento dos dados por semana

In [ ]:
semana_53 = df.loc[df['semana_do_ano'] == 53]
print(semana_53.head())

>Essa etapa de seleção de variáveis foi crucial para garantir que o modelo XGBoost fosse treinado apenas com as variáveis mais relevantes, maximizando assim seu desempenho e eficácia preditiva. Ao remover as variáveis que não contribuem significativamente para o modelo, também garantimos uma implementação mais enxuta e eficiente, o que é fundamental em ambientes de produção.

In [ ]:
df = df.drop(['date', 'dia', 'dia_do_ano', 'dia_da_semana', 'fim_de_semana', 'dia_do_ano_sin', 'dia_do_ano_cos', 'dia_da_semana_sin', 'trimestre', 'dia_da_semana_cos'], axis=1)

>Neste trecho de código, o DataFrame `df` está sendo reagrupado semanalmente com base em três colunas: `ano`, `semana_do_ano`, e `sku`. A partir deste agrupamento, várias agregações são realizadas nas colunas restantes, tais como calcular a média de preços e a soma total de itens vendidos durante cada semana, e determinar os valores modais para características específicas do produto e condições de mercado. O resultado é um novo DataFrame onde cada linha representa um `sku` específico em uma dada semana do ano, com variáveis agregadas que resumem as informações relevantes para aquele período.


In [ ]:
aggregations = {
    'unit_price': 'mean',
    'winning_price': 'mean',
    'items_sold': 'sum',
    'avg_website_visits_last_week': 'mean',
    'stock_qty': lambda x: x.mode().iloc[0],
    'dolar': 'mean',
    'selic': 'mean',
    'is_national': lambda x: x.mode().iloc[0],
    'can_provide_material': lambda x: x.mode().iloc[0],
    'shipment_type_próprio': lambda x: x.mode().iloc[0],
    'price_status_Ganhando': lambda x: x.mode().iloc[0],
    'price_status_Perdendo': lambda x: x.mode().iloc[0],
    'price_status_Único Disponível': lambda x: x.mode().iloc[0],
    'anchor_category': lambda x: x.mode().iloc[0],
    'product_department': lambda x: x.mode().iloc[0],
    'product_category': lambda x: x.mode().iloc[0],
    'sku_color': lambda x: x.mode().iloc[0],
    'mes': lambda x: x.mode().iloc[0],
}

df = df.groupby(['ano', 'semana_do_ano', 'sku']).agg(aggregations).reset_index()

##### Remoção de Outliers:

Nesta seção do código ocorre a remoção de outliers, um dos modos de tratamento de dados que pode ser aplicado para melhorar a performance do modelo. A remoção de outliers é uma técnica que consiste em identificar e remover valores extremos que podem causar distorções nos resultados. A remoção de outliers é uma técnica que consiste em identificar e remover valores extremos que podem causar distorções nos resultados.

O método ``IQR`` (Interquartile Range) é um método estatístico que utiliza a amplitude interquartil para determinar outliers. A amplitude interquartil é a diferença entre o terceiro e o primeiro quartil. O primeiro quartil é o valor que deixa 25% dos dados abaixo e 75% acima, enquanto o terceiro quartil deixa 75% dos dados abaixo e 25% acima.

In [ ]:
df.columns

In [ ]:
q1 = df['items_sold'].quantile(0.25)
q3 = df['items_sold'].quantile(0.75)

iqr = q3 - q1

lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

df = df[(df['items_sold'] >= lower_bound) & (df['items_sold'] <= upper_bound)]

A partir do ``IQR``, outliers são identificados e removidos do dataset. Esta medida de dispersão estatística proporciona a amplitude interquartil, facilitando a detecção e filtragem de valores extremos. A aplicação desse método resultou em uma redução aproximada de 9% nos registros originais.

No código acima, é possível observar a aplicação do método ``IQR`` para a remoção de outliers, utilizando 1.5 como fator multiplicador da amplitude interquartil, que é representa a diferença entre o terceiro e o primeiro quartil.

A coluna "items_sold" foi utilizada como variável alvo, pois é a variável que se deseja-se prever, ou seja, as vendas dos SKUs. As demais colunas foram utilizadas como variáveis independentes.

##### Divisão dos datasets

A seguir, foram preparados os conjuntos de dados para treinamento e teste. Em ambos os dataests foram removidas colunas irrelevantes para a modelagem e foram separadas as features (``X``) dos targets (``y``), para cada dataset.

In [ ]:
df.columns

Atribuindo mais peso aos SKUs com price_status ganhando, visto que a chance

In [ ]:
X = df.drop(['items_sold'], axis=1)
y = df['items_sold']

In [ ]:
X = df.drop(['items_sold'], axis=1)
y = df['items_sold']

Após o tratamento inicial dos dados, os conjuntos de features (``X``) e target (``y``) foram preparados, para os dois dataframes. Os atributos relacionados a 'revenue', 'items_sold', 'date' e alguns outros foram removidos, pois eles não seriam úteis na etapa de treinamento do modelo.

Posteriormente, os dados foram divididos em conjuntos de treino e teste utilizando uma proporção de 70% para treino e 30% para teste.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Random search

##### Ajuste dos Hiperparâmetros usando Random Search
>O método ``Random Search`` é utilizado na otimização de hiperparâmetros. Embora seja possível abranger um vasto espaço de hiperparâmetros, por uma questão de viabilidade de capacidade computacional, a busca foi restrita ao seguinte intervalo.

In [ ]:
param_dist = {
    'n_estimators': np.arange(50, 1000, 50),
    'max_depth': np.arange(3, 15),
    'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5]
}

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror')

Aqui é feita a busca pelos melhores hiperparâmetros

random_search = RandomizedSearchCV(estimator=xgb_reg, param_distributions=param_dist, n_iter=15, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)

best_params = random_search1.best_params_
print("Melhores hiperparâmetros:", best_params)

>O resultado encontrado pelo `Random Search` em ambos os casos foi:
~~~python
{
 'n_estimators': 350,
 'max_depth': 7,
 'learning_rate': 0.2
}
~~~

## Modelo XGBoost

### Treinamento com retirada de Outliers

In [ ]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators = 350, max_depth = 7, learning_rate = 0.2)

In [ ]:
xgb_reg.fit(X_train, y_train)

In [ ]:
y_pred = xgb_reg.predict(X_test)

Para as métricas, foram escolhidos o ``MAE``, o ``MSE``, o ``RMSE`` e o ``R²``. O ``MAE`` é a média da diferença absoluta entre o valor real e o valor previsto, o ``MSE`` é a média dos erros quadráticos, o ``RMSE`` é a raiz quadrada do erro quadrático médio, ou seja, é a raiz quadrada da média dos erros ao quadrado e o ``R²`` é uma medida estatística que representa a proporção da variância para uma variável dependente que é explicada por uma variável independente ou variáveis independentes em um modelo de regressão.

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f"Erro Médio Absoluto (MAE): {mae}")

mse = mean_squared_error(y_test, y_pred)
print(f"Erro Quadrático Médio (MSE): {mse}")

rmse = np.sqrt(mse)
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse}")

r2 = r2_score(y_test, y_pred)
print(f"Coeficiente de Determinação (R²): {r2}")

Semanal - Os resultados obtidos são:

| Métrica                               | Valor                  |
|---------------------------------------|------------------------|
| Erro Médio Absoluto (MAE)             | 3.637822905919477     |
| Erro Quadrático Médio (MSE)           | 28.91451108852052     |
| Raiz do Erro Quadrático Médio (RMSE)  | 5.377221502646187     |
| Coeficiente de Determinação (R²)      | 0.7247192138758027     |

## Discussão dos Resultados

-Coeficiente de Determinação (R²): O coeficiente de determinação, ou R², é uma medida que varia de 0 a 1 e indica o quanto nosso modelo é capaz de explicar a variação nos dados. No caso, nosso modelo XGBoost possui um R² de 0.72, o que significa que ele consegue explicar aproximadamente 72% da variação nas vendas dos produtos, o que é uma boa performance.

-Erro Médio Absoluto (MAE): O erro médio absoluto, ou MAE, é uma métrica que nos diz quanto, em média, nosso modelo está errando nas previsões de vendas por SKU. No nosso caso, o MAE é de 3.63. Isso significa que, em média, nosso modelo está errando cerca de 3.63 unidades nas previsões de vendas por SKU.

-Erro Quadrático Médio (MSE): O erro quadrático médio, ou MSE, é outra métrica que avalia o erro das previsões, mas ao elevar os erros ao quadrado, ele dá mais peso a erros maiores. O MSE obtido foi de 28.91. Isso significa que, em média, nosso modelo tem um erro quadrático de 28.91 unidades nas previsões de vendas por SKU.

-Raiz do Erro Quadrático Médio (RMSE): O RMSE é a raiz quadrada do MSE e é uma métrica que nos fornece uma medida do erro em unidades originais, o que facilita a interpretação. No nosso caso, o RMSE é de 5.37, o que indica que, em média, nosso modelo erra cerca de 5.37 unidades nas previsões de vendas por SKU.

## Explicabilidade do modelo

A explicabilidade do modelo tem o objetivo de transformar o funcionamento interno de um modelo em um formato compreensível para os seres humanos, de modo que possamos entender por que o modelo faz as previsões ou decisões que faz. Para realizar a explicabilidade, utiliza-se a técnica SHAP (SHapley Additive exPlanations), em que o modelo realiza todas as combinação de todas as possíveis permutações de características, calculando quanto cada característica contribuiu para a diferença entre a previsão do modelo e o valor médio das previsões. 

In [ ]:
!python -m pip install shap

In [ ]:
import shap

Aplicando a técnica 

In [ ]:
explainer = shap.Explainer(xgb_reg)

In [ ]:
shap_values = explainer.shap_values(X_test)

Utiliza-se a média ponderada para calcular cada contribuição e o resultado se dá pela amostra individual do impacto de cada variável com base na variação dos valores da variável. No gráfico abaixo, apresenta-se tons de azul para menores valores e tons de vermelho para valores mais altos da variável no modelo.

In [ ]:
shap.summary_plot(shap_values, X_test)

# Previsão para 90 dias

Nessa etapa, foi realizada uma previsão para projetar as vendas das próximas 13 semanas, cerca de 90 dias, utilizando os dados da última semana disponível de 2023. Esta simulação demonstra um possível uso do modelo em um cenário real, servindo como uma prévia de como a Mobly pode aproveitar o modelo para antecipar tendências do estoque.

In [ ]:
last_week_data = X[(X['ano'] == 2023) & (X['semana_do_ano'] == 27)]

predictions = []

for index, row in last_week_data.iterrows():
    for i in range(1, 14):  
        week_data = row.copy()
        week_data['semana_do_ano'] += i
        
        input_data = pd.DataFrame([week_data])
    
        prediction = xgb_reg.predict(input_data)
        
        predictions.append({
            'sku': week_data['sku'],
            'ano': week_data['ano'],
            'semana_do_ano': week_data['semana_do_ano'],
            'prediction': prediction[0]
        })

In [ ]:
predictions_df = pd.DataFrame(predictions).sort_values(by=['semana_do_ano'])
predictions_df['sku'] = predictions_df['sku'].map(decoding_dict['sku'])
predictions_df.to_csv('predictions.csv', index=False)

Amostra da previsão para 90 dias com 10 SKUs na tabela abaixo:

| sku                        | ano  | semana_do_ano | previsão |
|----------------------------|------|---------------|------------|
| AC967UP19DSYMOB-181803     | 2023 | 28            | 0.881078   |
| MO173TA90IDTMOB-922935     | 2023 | 28            | 6.531820   |
| LI729RA09NSOMOB-772010     | 2023 | 28            | 2.497056   |
| CO184TA82PNTMOB-773204     | 2023 | 28            | 4.215886   |
| MO173TA46OKJMOB-772441     | 2023 | 28            | 1.505563   |
| LI729RA82QGZMOB-773692     | 2023 | 28            | 0.467013   |
| MO173UP09VXAMOB-794589     | 2023 | 28            | 3.976960   |
| LI582UP86DRHMOB-199503     | 2023 | 28            | 15.676572  |
| CO742UP21LWKMOB-600720     | 2023 | 28            | 2.161284   |
| OR067MA27ZQYMOB-557817     | 2023 | 28            | 6.934310   |


# Armazenamento do modelo

In [ ]:
dump(xgb_reg, 'modelo_xgboost.pkl')

# Considerações Finais

Após uma análise comparativa dos modelos candidatos, ficou evidente que o XGBoost se destacou, superando os outros em relação às métricas de avaliação e à precisão nas previsões de vendas. Com base em seus resultados, decidimos escolher o XGBoost como a solução final.